# SpectralIndices.jl

This tutorial closely follows the SpectralIndices.jl documentation. It provides a gentle walk through of all the features of the package in an incremental fashion.

If you are running this notebbok on Google colab please run this cell below. It contains all the packages and Julia installation we need for the workshop. After running the cell please reload the page to load the new Julia runtime. If this doens't work, go to Runtime->Change runtime type and change from Python3 to julia 1.10.2

In [1]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.10.2" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools SpectralIndices YAXArrays DataFrames DimensionalData"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

In [1]:
using SpectralIndices

## Introduction to Indices Calculation
We want to calculate the NDVI. Let's create the variables needed for this

In [2]:
nir = 6723
red = 1243

1243

Now let's explore the NDVI by simply calling it in the repl


In [3]:
NDVI

NDVI: Normalized Difference Vegetation Index
* Application Domain: vegetation
* Bands/Parameters: Any["N", "R"]
* Formula: (N-R)/(N+R)
* Reference: https://ntrs.nasa.gov/citations/19740022614


as we can see all useful information is already there!
What's more is that this struct also acts as a callable function

In [ ]:
NDVI(Float32, nir, red)

0.6879236756213909

Pretty neat stuff, although I wouldnt' recommend using this as your primary mode of calculating indices. If you still want to use it make sure the order of the parameters
matches how they appear in the `bands` field of the index:

In [ ]:
NDVI.bands


2-element Vector{Any}:
 "N"
 "R"

A more flexible way, and the suggested approach, to calculate indices is through the `compute_index` function. This function accepts either the `SpectralIndex` struct or the spectral index name as input and parameters as either a dictionary or keyword arguments:

In [ ]:
compute_index("NDVI"; N=nir, R=red)

0.6879236756213909

In [ ]:
compute_index(NDVI; N=nir, R=red)

0.6879236756213909

In [ ]:
params_nr = Dict(
    "N" => nir,
    "R" => red
)

Dict{String, Int64} with 2 entries:
  "N" => 6723
  "R" => 1243

In [ ]:
ndvi_stpr = compute_index("NDVI", params_nr)

0.6879236756213909

In [ ]:
compute_index(NDVI, params_nr)

0.6879236756213909

### Quick sidenote on implementation details

In [ ]:
function ndvi_funcstring_eval(N, R; string_formula = "(N-R)/(N+R)")
    formula_with_values = replace(string_formula, "N" => "($N)", "R" => "($R)")
    expr = Meta.parse(formula_with_values)
    result = eval(expr)
    return result
end

function ndvi_funcstring_il(N, R; string_formula = "(N-R)/(N+R)")
    func_str = "f(N, R) = $string_formula"
    expr = Meta.parse(func_str)
    eval(expr)
    result = Base.invokelatest(f, N, R)
    return result
end

ndvi_pure(N, R) = (N-R)/(N+R)

ndvi_pure (generic function with 1 method)

In [ ]:
using BenchmarkTools

In [ ]:
@benchmark ndvi_funcstring_eval(0.2, 0.1)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  146.405 μs …  10.103 ms  ┊ GC (min … max): 0.00% … 95.94%
 Time  (median):     157.544 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   172.994 μs ± 150.127 μs  ┊ GC (mean ± σ):  1.03% ±  1.35%

  ▅██▇▇▇▆▆▅▄▃▃▃▃▂▃▂▂▁▁▁                                         ▃
  ██████████████████████▇▇▇████▇▇▆▇▇█▆█▇████▇████▇█▇▇▇▇▆▆▇▅▃▄▁▅ █
  146 μs        Histogram: log(frequency) by time        309 μs <

 Memory estimate: 10.03 KiB, allocs estimate: 159.

In [ ]:
@benchmark ndvi_funcstring_il(0.2, 0.1)

BenchmarkTools.Trial: 577 samples with 1 evaluation.
 Range (min … max):  5.900 ms … 24.584 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     8.285 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   8.620 ms ±  1.867 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

            ▅▃█▃▃▄▃▂▂                                         
  ▅█▅▃▅▄▄▃▄███████████▅▃▄▄▅▅█▆▅▆▃▅▃▃▃▂▃▃▃▃▁▂▂▂▃▁▁▂▂▂▁▁▁▂▁▁▂▃ ▃
  5.9 ms         Histogram: frequency by time        14.8 ms <

 Memory estimate: 62.32 KiB, allocs estimate: 945.

In [ ]:
@benchmark ndvi_pure(0.2, 0.1)

BenchmarkTools.Trial: 10000 samples with 1000 evaluations.
 Range (min … max):  1.882 ns … 39.608 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.919 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   2.279 ns ±  0.990 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▅          ▂▄▄▄▄▄▄▃▃▃▃▃▃▃▂▂▁▁                             ▂
  ██▆▅▁▃▃▃▁▁▁▆███████████████████▇▇▇▇█▇▇▇▇▆▇▇▆▆▄▅▄▄▁▃▄▄▅▄▄▅▅ █
  1.88 ns      Histogram: log(frequency) by time     3.78 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

As we can see the pure julia function as virtually no overhead, showing
- zero memory allocations
- nanosecond-level execution time

The function based on eval shows:
- efficient runtime evaluation of the function
- usable microsecond range
- this method is not usable in the context of a package, due to the world age problem

The function with invokelast solves the world age problem but:
- Has the highest overhead in terms of both execution time and memory usage
- Scales poorly for large quantites of data

## Extending the examples: a new index and floats

In [ ]:
SAVI

SAVI: Soil-Adjusted Vegetation Index
* Application Domain: vegetation
* Bands/Parameters: Any["L", "N", "R"]
* Formula: (1.0+L)*(N-R)/(N+R+L)
* Reference: https://doi.org/10.1016/0034-4257(88)90106-X


In [ ]:
SAVI.bands

3-element Vector{Any}:
 "L"
 "N"
 "R"

The `L` parameter is new in this example.
Thankfully, SpectralIndices.jl provides a list of constant values handy that we can leverage in this situation:

In [ ]:
constants["L"]

L: Canopy background adjustment
* Description: Canopy background adjustment
* Standard: L
* Default value: 1.0
* Current value: 1.0


Now that we know what L is, let's proceed with the calculation.


SAVI needs input data to be between -1 and 1

In [ ]:
scaled_nir = nir/10000
scaled_red = red/10000

0.1243

In [ ]:
compute_index("SAVI", Dict(
    "N" => scaled_nir,
    "R" => scaled_red,
    "L" => 0.5)
)

0.6339657565941694

In [ ]:
compute_index("SAVI"; N=scaled_nir, R=scaled_red, L=0.5)

0.6339657565941694

Now that we have introduced multiple indices let's see how we can compute multiple indices at the same time:

In [ ]:
params_mi = Dict(
    "N" => nir,
    "R" => red,
    "L" => 0.5
)

Dict{String, Real} with 3 entries:
  "N" => 6723
  "L" => 0.5
  "R" => 1243

In [ ]:
compute_index(["NDVI", "SAVI"], params_mi)

2-element Vector{Any}:
 0.6879236756213909
 1.0318207493880625

In [ ]:
compute_index(["NDVI", "SAVI"]; N=scaled_nir, R=scaled_red, L=0.5)

2-element Vector{Any}:
 0.687923675621391
 0.6339657565941694

All of this can be extended to vectors as well:

In [ ]:
params_miv = Dict(
    "N" => fill(nir, 10),
    "R" => fill(red, 10),
    "L" => fill(0.5, 10)
)

Dict{String, Vector} with 3 entries:
  "N" => [6723, 6723, 6723, 6723, 6723, 6723, 6723, 6723, 6723, 6723]
  "L" => [0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
  "R" => [1243, 1243, 1243, 1243, 1243, 1243, 1243, 1243, 1243, 1243]

In [ ]:
compute_index(["NDVI", "SAVI"], params_miv)

2-element Vector{Any}:
 [0.6879236756213909, 0.6879236756213909, 0.6879236756213909, 0.6879236756213909, 0.6879236756213909, 0.6879236756213909, 0.6879236756213909, 0.6879236756213909, 0.6879236756213909, 0.6879236756213909]
 [1.0318207493880625, 1.0318207493880625, 1.0318207493880625, 1.0318207493880625, 1.0318207493880625, 1.0318207493880625, 1.0318207493880625, 1.0318207493880625, 1.0318207493880625, 1.0318207493880625]

We can use the same params to calculate single indices.
The additional bands are just going to be ignored:

In [ ]:
compute_index("NDVI", params_miv)

10-element Vector{Float64}:
 0.6879236756213909
 0.6879236756213909
 0.6879236756213909
 0.6879236756213909
 0.6879236756213909
 0.6879236756213909
 0.6879236756213909
 0.6879236756213909
 0.6879236756213909
 0.6879236756213909

In [ ]:
compute_index("SAVI", params_miv)

10-element Vector{Float64}:
 1.0318207493880625
 1.0318207493880625
 1.0318207493880625
 1.0318207493880625
 1.0318207493880625
 1.0318207493880625
 1.0318207493880625
 1.0318207493880625
 1.0318207493880625
 1.0318207493880625

Using kwargs is also still straightforward as before

In [ ]:
compute_index(["NDVI", "SAVI"];
    N=fill(nir, 10),
    R=fill(red, 10),
    L=fill(0.5, 10))

2-element Vector{Any}:
 [0.6879236756213909, 0.6879236756213909, 0.6879236756213909, 0.6879236756213909, 0.6879236756213909, 0.6879236756213909, 0.6879236756213909, 0.6879236756213909, 0.6879236756213909, 0.6879236756213909]
 [1.0318207493880625, 1.0318207493880625, 1.0318207493880625, 1.0318207493880625, 1.0318207493880625, 1.0318207493880625, 1.0318207493880625, 1.0318207493880625, 1.0318207493880625, 1.0318207493880625]

## Using DataFrames.jl

In [ ]:
using DataFrames

Of course you probably will not be using single values to calculate your indices!
This section illustrates how you can use SpectralIndices in conjuction with DataFrames.jl

Let's load some data to use in the examples:

In [ ]:
df = load_dataset("spectral", DataFrame)

Row,SR_B5,ST_B10,SR_B2,SR_B6,class,SR_B4,SR_B7,SR_B3,SR_B1
,Float64,Float64,Float64,Float64,String,Float64,Float64,Float64,Float64
1,0.269054,297.328,0.100795,0.306206,Urban,0.165764,0.251949,0.132227,0.08985
2,0.281264,297.108,0.08699,0.267596,Urban,0.160979,0.217917,0.124404,0.0738588
3,0.28422,297.436,0.0860275,0.258384,Urban,0.140203,0.200098,0.120994,0.0729375
4,0.254479,297.204,0.103916,0.25958,Urban,0.163976,0.216735,0.135981,0.0877325
5,0.269535,297.098,0.109306,0.273234,Urban,0.18126,0.219554,0.15035,0.0905925
6,0.277153,298.212,0.107148,0.32954,Urban,0.19754,0.253929,0.152303,0.0927237
7,0.26563,298.402,0.100396,0.271721,Urban,0.170026,0.222331,0.135885,0.087375
8,0.294478,298.448,0.102032,0.318224,Urban,0.186224,0.238515,0.139749,0.0901113
9,0.293446,297.604,0.146239,0.29753,Urban,0.210094,0.237649,0.179624,0.132846


Each column of this dataset is the Surface Reflectance from Landsat 8 for 3 different classes. The samples were taken over Oporto. The data is taken from [spyndex](https://spyndex.readthedocs.io/en/latest/tutorials/pandas.html). This dataset specifically contains three different classes:


In [ ]:
unique(df[!, "class"])

3-element Vector{String}:
 "Urban"
 "Water"
 "Vegetation"

so to reflect that we are going to calculate three different indices: `NDVI` for `vegetation`, `NDWI` for `water` and `NDBI` for `urban`.
Let's see what bands we need:

In [ ]:
NDVI.bands

2-element Vector{Any}:
 "N"
 "R"

In [ ]:
NDWI.bands

2-element Vector{Any}:
 "G"
 "N"

In [ ]:
NDBI.bands

2-element Vector{Any}:
 "S1"
 "N"

We need Green, Red, NIR and SWIR1 bands.

Since the `compute_index` expects the bands to have the same name as the have in the `bands` field we need to select the specific columns that we want out of the dataset and rename them.


We can do this easily with `select`:

In [ ]:
params_df = select(df, :SR_B3=>:G, :SR_B4=>:R, :SR_B5=>:N, :SR_B6=>:S1)


SyntaxError: invalid syntax (<ipython-input-1-4bf9f03e2012>, line 1)

Now our dataset is ready, and we just need to call the `compute_index` function

In [ ]:
compute_index(["NDVI", "NDWI", "NDBI"], params_df)

Another way to obtain this is to feed single `DataFrame`s as kwargs.


In [ ]:
compute_index(["NDVI", "NDWI", "NDBI"];
    G = select(df, :SR_B3=>:G),
    N = select(df, :SR_B5=>:N),
    R = select(df, :SR_B4=>:R),
    S1 = select(df, :SR_B6=>:S1)
)

Alternatively you can define a `Dict` for the indices from the `DataFrame`

In [ ]:
params_dfdf = Dict(
    "G" => df[!, "SR_B3"],
    "N" => df[!, "SR_B5"],
    "R" => df[!, "SR_B4"],
    "S1" => df[!, "SR_B6"]
)

In [ ]:
compute_index(["NDVI", "NDWI", "NDBI"], params_dfdf)


Again, we can just feed the single dataframes as kwargs

In [ ]:
compute_index(["NDVI", "NDWI", "NDBI"];
    G = df[!, "SR_B3"],
    N = df[!, "SR_B5"],
    R = df[!, "SR_B4"],
    S1 = df[!, "SR_B6"]
)

The difference is that using these two approaches will result in an array

Using YAXArrays.jl

In [ ]:
using YAXArrays, DimensionalData

As before let's load some data

In [ ]:
yaxa = load_dataset("sentinel", YAXArray)

We have a `YAXArray` object with three dimensions: `bands`, `x` and `y`.
Each band is one of the 10 m spectral bands of a Sentinel-2 image.
Data again taken from spyndex


The data is stored as `Int64`, so let us convert it to `Float` and rescale it:

In [ ]:
scaled_yaxa = yaxa./10000

Now let's compute the NDVI for this dataset!

In [ ]:
ndvi = compute_index("NDVI";
    N=scaled_yaxa[bands = At("B08")],
    R=scaled_yaxa[bands = At("B04")]
)

Alternatively we can build a custom YAXArrays and feed that into the function

In [ ]:
index_R = findfirst(scaled_yaxa.bands.val .== "B04")

index_N = findfirst(scaled_yaxa.bands.val .== "B08")

new_bands_dim = Dim{:Variables}(["R", "N"])

nr_data = cat(scaled_yaxa[:, :, index_R], scaled_yaxa[:, :, index_N], dims=3)

new_yaxa = YAXArray((scaled_yaxa.x, scaled_yaxa.y, new_bands_dim), nr_data)

ATT! Please notice how the `Dim` is called `Variables`.

This is needed for the internal computation to work properly.

In [ ]:
compute_index("NDVI", new_yaxa)

## Computing Kernels
We want to compute the kNDVI for a `YAXArray`.

In [ ]:
kNDVI

As we see from `bands` we need the `kNN` and `kNR`.
In order to compute these values SpectralIndices.jl provides a `compute_kernel` function.

If you are curious about the `kNDVI` remember that you always have the source of the index in the `reference` field:


In [ ]:
kNDVI.reference

Now that we are up to speed with the literature let's get to the computin'

In [ ]:
knn = YAXArray((scaled_yaxa.x, scaled_yaxa.y), fill(1.0, 300, 300))

In [ ]:
knr = compute_kernel(
    RBF;
    a = Float64.(scaled_yaxa[bands = At("B08")]),
    b = Float64.(scaled_yaxa[bands = At("B04")]),
    sigma = scaled_yaxa[bands = At("B08")].+scaled_yaxa[bands = At("B04")] ./ 2
)

As before, you can decide to build an `YAXArray` and feed that to the `compute_kernel` function if you prefer:


In [ ]:
a = Float64.(scaled_yaxa[bands = At("B08")])

b = Float64.(scaled_yaxa[bands = At("B04")])

sigma = scaled_yaxa[bands = At("B08")].+scaled_yaxa[bands = At("B04")] ./ 2

kernel_dims = Dim{:Variables}(["a", "b", "sigma"])

params_ks = concatenatecubes([a, b, sigma], kernel_dims)

In [ ]:
compute_kernel(RBF, params_ks)

We can finally compute the kNDVI:

In [ ]:
kndvi = compute_index("kNDVI"; kNN = knn, kNR=knr)

## Additional info
I will be presenting SpectralIndices.jl in depth at the conference track, tomorrow 03.07.2024! I will go more into the structure of the package and how you can leverage it to improve your workflow with a focus on machine learning applications.

Additionally you can find the software at **https://github.com/awesome-spectral-indices/SpectralIndices.jl** and related paper at https://isprs-archives.copernicus.org/articles/XLVIII-4-W12-2024/89/2024/isprs-archives-XLVIII-4-W12-2024-89-2024.pdf

Thank you all for your attention!